<div class="alert alert-block alert-info">
<b>Information zu Notebooks:</b>
  In diesem Notebook wird sowohl Text als auch der Python Code dargestellt. Der Python Code mit dem Sympy Tool ist immer in grauen Boxen hinterlegt und wird schrittweise durchgeführt. Die Textpassagen sind weiß hinterlegt. 
Damit die Darstellung übersichtlich ist, wurde die Ausgabe mit dem Befehl <i>print_latex</i> in Latex gemacht, die aber zur besseren Lesbarkeit wieder auskommentiert worden ist. 

<b>Hinweis: Qubits mathematisch-physikalischer Darstellung versus IBM Quantum/Qiskit</b>

Die mathematisch-physikalische Bra-Ket Darstellungsweise von Qubits, welche auch das Tool Sympy in diesem Notebook verwendet, beginnt mit dem niedrigsten Qubit rechts. Anders ist die Darstellung bei IBM Quantum, das auf dem Programmcode Qiskit basiert. Qiskit fängt links mit dem niedrigsten Qubit an anstelle von rechts. Daher werden die Ergebnisse in Qiskit bei IBM Quantum beispielsweise als $|10\rangle$ dargestellt. Dies entspräche der mathematisch-physikalische Darstellung $|01\rangle$. 

Ausführlich erklärt ist dieser Unterschied in der Dokumentation zu Quiskit unter folgendem Link (abgerufen am 02.11.2023):

https://qiskit.org/documentation/tutorials/circuits/1_getting_started_with_qiskit.html
       
    
</div>

## Simulation des Grover-Algorithmus mit zwei Qubits

Im Folgenden wird der Grover-Algorithmus mit zwei Qubits durchgeführt (Darstellung als Jupyter Notebook, programmmiert mit Python). Sodann wird im Rahmen des Orakels der Status  $ {\left|01\right\rangle } $markiert und im Ergebnis gefunden werden. Der Status  $ {\left|01\right\rangle } $ ist insofern interessant, da hierfür Quantengatter mit komplexen Zahlen verwendet werden. Insofern erfolgt die Rechnung mit Matrizen mit komplexen Zahlen.

Zur besseren Veranschaulichung wird jeweils zum einen die mathematische Vorgehensweise kurz beschrieben und zum anderen die  Berechnung in Matrizenschreibweise dargestellt. 

Für die Berechnung wird das Python Tool Sympy verwendet (grau hinterlegter Bereich), das eine symbolhafte Berechnung von Qubits und deren Matrizen erlaubt. 

Der Grover-Algorithmus wird nachfolgend in drei Schritten, erstens der Superposition, zweitens dem Orakel und drittens der Inversionsspiegelung, dargestellt. 

In [30]:
import sympy as sp

# für Latex Darstellung
from IPython.display import Latex
from IPython.display import display, Math
#Sympy import
from sympy import *
from sympy.physics.quantum.qubit import IntQubit
from sympy.physics.quantum.qubit import Qubit
from sympy.physics.quantum.qubit import qubit_to_matrix
from sympy.physics.quantum.qubit import matrix_to_qubit
from sympy.physics.quantum.qubit import measure_all
from sympy.physics.quantum import TensorProduct

 ## 1. Schritt:  Superposition
### 1.1 Initialisierung von zwei Qubits
Als erstes werden zwei Qubits im Zustand $$\psi\ = {\left|00\right\rangle } $$ definiert:

In [14]:
q1 = IntQubit(Qubit('00'))
# print_latex(Qubit(q1))

Alternativ kann man die Qubits wie folgt in Vektorschreibweise darstellen: 
$$\psi\ = \left[\begin{matrix}1\\0\\0\\0\end{matrix}\right]$$

In [15]:
psi=qubit_to_matrix(q1) # damit als Vektor
#print_latex(qubit_to_matrix(q1))

Dies entspricht folgender Quantenschaltung:

<img src="image/01_Qubits.png">


### 1.2 Durchführung der Superposition
Um die zwei Qubits in Superposition zu bringen, benötigen man das Hadamard-Gatter: 
$$ H = \left[\begin{matrix}\frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2}\\\frac{\sqrt{2}}{2} & - \frac{\sqrt{2}}{2}\end{matrix}\right]$$


In [16]:
H =Matrix([[1/sqrt(2),1/sqrt(2)],[1/sqrt(2),-1/sqrt(2)]])
#print_latex(H)

Die Superposition erfolgt konkret, indem man mit Hilfe des Tensorproduktes zwei Hadamard-Gatter auf die beiden Qubits anwendet:
$$H^{\otimes 2} = H\otimes H $$ 

In [17]:
H2=TensorProduct(H,H)
#print_latex(H2)

Das Ergebnis ist eine 4x4 Matrix:

$$ H^{\otimes 2}  = \left[\begin{matrix}\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & - \frac{1}{2} & \frac{1}{2} & - \frac{1}{2}\\\frac{1}{2} & \frac{1}{2} & - \frac{1}{2} & - \frac{1}{2}\\\frac{1}{2} & - \frac{1}{2} & - \frac{1}{2} & \frac{1}{2}\end{matrix}\right]$$


Diese wird anschließend mit den Qubits multiplizert. Als Ergebnis erhält man alle Qubits, gekennzeichnet durch den Ergebnisvektor 
$ |s\rangle $ in der Superposition.  
 
**1. Schritt:**  Zwei Qubits in die Superposition bringen: $$ |s\rangle= H^{\otimes 2} |\psi\rangle =  $$ 

$$  \left[\begin{matrix}\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & - \frac{1}{2} & \frac{1}{2} & - \frac{1}{2}\\\frac{1}{2} & \frac{1}{2} & - \frac{1}{2} & - \frac{1}{2}\\\frac{1}{2} & - \frac{1}{2} & - \frac{1}{2} & \frac{1}{2}\end{matrix}\right] \cdot \left[\begin{matrix}1\\0\\0\\0\end{matrix}\right] = $$

$$ |s\rangle  = \left[\begin{matrix}\frac{1}{2}\\\frac{1}{2}\\\frac{1}{2}\\\frac{1}{2}\end{matrix}\right] $$


In [18]:
superposition=H2*psi
#print_latex(superposition)

Dies entspricht der folgenden Schaltung: 

<img src="image/02_Superposition.png">


## 2. Schritt: Orakel 

Als Vorbereitung für die spätere Durchführung des Orakels sind weitere Quantengatter erforderlich. 

### 2.1 Definition weiterer Gatter
Für die Durchführung des Orakels werden drei weitere Gatter bzw. deren Ausprägungen als Matrizen benötigt:  
**1. Gatter: Einheitsgatter $I$**: Dieses Gatter hat die folgende Darstellung: $$ I = \left[\begin{matrix}1 & 0\\0 & 1\end{matrix}\right] $$  
Es wird in einem Quantenschaltkreis verwendet, wenn auf ein Qubit kein anderes, spezielleres Gatter angewandt wird. 

**2. Gatter: Das S-Gate**: Dieses Gatter dreht die Phase um π/2 und hat damit einen komplexen Anteil in der mathematischen Ausprägung: $$ S=  \left[\begin{matrix}1 & 0\\0 & i\end{matrix}\right] $$
In einem Quantenschaltkreis wird es durch ein S dargestellt und hat folgende Symbolausprägung:

<img src="image/s-gate.png">
 
**3. Gatter: CZ-Gatter**: Dieses sogenannte "Controlled-Z-Gatter" benötigt man für den Operator $U_w$. Es wirkt als Gatter über zwei Qubits und muss als Matrix eine 4x4 Darstellung haben.

$$ CZ =\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & -1\end{matrix}\right] $$
Es wird im Quantenschaltkreis als Strich mit Verbindungspunkten über zwei Qubits dargestellt.

<img src="image/cz-gate.png">

**4. Gatter: X-Gatter**: Dies ist das sogenannte "Pauli-X-Gatter". 
$$ \left[\begin{matrix}0 & 1\\1 & 0\end{matrix}\right] $$
Es wird durch ein + dargestellt.

<img src="image/x-gate.png">

**5. Gatter: Z-Gatter**: Schließlich benötigt man noch das sogenannte "Z-Gatter". 
$$ \left[\begin{matrix}1 & 0\\0 & -1\end{matrix}\right] $$
Es wird durch ein Z dargestellt. 

<img src="image/z-gate.png">


In [19]:
I= Matrix([[1,0],
         [0,1,]])
#print_latex(I)
S=Matrix([[1,0],
         [0,sp.I]])
#print_latex(S)
CZ=Matrix([[1,0,0,0],
       [0,1,0,0],
       [0,0,1,0],
       [0,0,0,-1]])
#print_latex(CNOT)
X=Matrix([[0,1],
          [1,0]])
Z=Matrix([[1,0],
          [0,-1]])


### 2.2  Das Orakel

Nun wird das Orakel angewendet, um damit die Position  $ {\left|01\right\rangle } $ zu markieren. Die Anwendung des Orakels wird typischerweise durch den Operator $ U_w $ gekennzeichnet. 

**2. Schritt:**  Anwendung des Orakels $ U_w $ auf $ |s\rangle  $    $$ U_w |s\rangle = \psi_2 $$    

Damit  $ {\left|01\right\rangle } $ markiert wird, braucht man eine Matrix, die wie folgt aussieht, damit  $ {\left|01\right\rangle } $ in der Amplitude gespiegelt wird:
$$ U_w = \left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right] $$


### 2.3 Erzeugen des Orakel Operators $U_w$

Um diese Matrize in der Quanteschaltung zu erzeugen, muss mathematisch folgende Operation mit den Gattern durchgeführt werden. 
$$ U_w = (I \otimes S)\cdot(CZ)\cdot (I \otimes S) $$

Nachvollziehbarer wird es in folgenden Einzelschritten:
$$ 
(I \otimes S ) =\left[\begin{matrix}1 & 0\\0 & 1\end{matrix}\right] \otimes 
\left[\begin{matrix}1 & 0\\0 & i\end{matrix}\right] = 
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & i & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & i\end{matrix}\right] 
$$ 
Dann ergibt sich folgendes:
$$ (I \otimes S )\cdot (CZ) = 
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & i & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & i\end{matrix}\right]  \cdot 
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & -1\end{matrix}\right] =
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & i & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & - i\end{matrix}\right]
$$
Schließlich erhält man:
$$ 
U_w = (I \otimes S)\cdot(CZ)\cdot(I \otimes S) = 
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & i & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & - i\end{matrix}\right] \cdot
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & i & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & i\end{matrix}\right] =
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

Das Ergebnis dieser Operation ist die gewünschte Matrix, 
$$
U_w = \left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$ 
die den Zustand  $ {\left|01\right\rangle }$  markieren wird.


In [29]:
#print_latex(TensorProduct(I,S))
#print_latex(TensorProduct(I,S)*CZ)
U_W=TensorProduct(I,S)*CZ*TensorProduct(I,S)
#print_latex(U_W)

Das finale Orakel $U_w$ sieht damit als Quantengatter wie folgt aus:

<img src="image/03_1_Oracle.png">


Jetzt wird $ U_w $ auf den Superpositionszustand $ |s\rangle $ angewandt. Damit erhält man das Orakel und den markierten Zustand 
bei $ {\left|01\right\rangle} $. Die gesuchte gespiegelte Amplitude ist nun negativ.
$$
\psi_2 = U_w |s\rangle  =  
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right] \cdot
\left[\begin{matrix}\frac{1}{2}\\\frac{1}{2}\\\frac{1}{2}\\\frac{1}{2}\end{matrix}\right] =
\left[\begin{matrix}\frac{1}{2}\\-\frac{1}{2}\\ \frac{1}{2}\\\frac{1}{2}\end{matrix}\right]
$$ . 


In [22]:
psi_2=U_W*superposition
#print_latex(psi_2)

In der Entwicklung des Quantengatters ist nun folgender Zustand erreicht:

<img src="image/03_oracle.png">


## 3. Schritt: Inversionsspiegelung

Mit den beiden Schritten   

**1. Schritt:**  Zwei Qubits in Superposition bringen: $$ H_2 |\psi\rangle = |s\rangle  $$
**2. Schritt:**  Anwendung des Orakels $ U_w $ auf $ |s\rangle $    $$ U_w |s\rangle  = \psi_2 $$    

íst der Vektor markiert und die gesuchte Amplitude gespiegelt. 
Nun folgt der nächste Schritt, nämlich  die Inversionsspiegelung. Hierbei wird die Amplitude verstärkt durch Spiegelung am Superpositionsvektor $ |s\rangle$. Da es vorliegend nur zwei Qubits sind, genügt eine Spiegelung, um zum Ergebnis zu kommen. 


**3. Schritt:**
Die Inversionsspiegelung erfolgt, indem $ U_S = 2|s\rangle  \langle s| -I $ ausgeführt wird. Da es sich dabei um eine Spiegelung an $|s\rangle $ handelt, kann man dies auf folgende Weise durchführen.
Da $H|s\rangle  = |0\rangle  $ ist und sich die Spiegelung durch Negierung der Orthogonalen ergibt, kann man das Hadamard-Gatter anwenden und anschließend die Orthogonalen zu $|0\rangle$ negieren. Zum Schluss wird wieder $H|s\rangle $ angewendet, so dass der Vektor $|\psi_2\rangle$ an $|s\rangle $ gespiegelt ist.

Der gesamte Operator wäre $$ U_S= H^{\otimes 2}\cdot(Z \otimes Z )\cdot(CZ) \cdot H^{\otimes 2} $$ 

wobei $ U_{S1}= (Z \otimes Z )(CZ) $ die Negierung der Orthogonalen zu $|0\rangle$ darstellt, die nun berechnet wird:
$$
(Z \otimes Z )= 
\left[\begin{matrix}1 & 1\\0 & -1\end{matrix}\right] \otimes 
\left[\begin{matrix}1 & 1\\0 & -1\end{matrix}\right] =
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & -1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

$$
U_{S1}=(Z \otimes Z )\cdot(CZ) =
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & -1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]\cdot
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & -1\end{matrix}\right] =
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & -1 & 0\\0 & 0 & 0 & -1\end{matrix}\right]
$$

Es wird ersichtlich, dass diese Matrix zu einer Negierung aller Zustände außer bei $|0\rangle$ führt.

Nachfolgend wird die Quantenschaltung dargestellt:

<img src="image/Drehung_Z.png">


In [24]:
U_s1=TensorProduct(Z,Z)*CZ
#print_latex(U_s1)

Damit ergibt sich folgendes: 
$$U_S=H^{\otimes 2}\cdot(Z \otimes Z )\cdot(CZ)\cdot H^{\otimes 2}=$$

$$ H^{\otimes 2}U_{s1}H^{\otimes 2} = 
\left[\begin{matrix}\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\
\frac{1}{2} & - \frac{1}{2} & \frac{1}{2} & - \frac{1}{2}\\
\frac{1}{2} & \frac{1}{2} & - \frac{1}{2} & - \frac{1}{2}\\
\frac{1}{2} & - \frac{1}{2} & - \frac{1}{2} & \frac{1}{2}\end{matrix}\right] \cdot 
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & -1 & 0\\0 & 0 & 0 & -1\end{matrix}\right]\cdot
\left[\begin{matrix}\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\
\frac{1}{2} & - \frac{1}{2} & \frac{1}{2} & - \frac{1}{2}\\
\frac{1}{2} & \frac{1}{2} & - \frac{1}{2} & - \frac{1}{2}\\
\frac{1}{2} & - \frac{1}{2} & - \frac{1}{2} & \frac{1}{2}\end{matrix}\right] =
$$

$$
U_S =
\left[\begin{matrix}- \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & - \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & \frac{1}{2} & - \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & - \frac{1}{2}\end{matrix}\right]
$$

In [26]:
U_s=H2*TensorProduct(Z,Z)*CZ*H2
#print_latex(U_s)

Um den Grover $G$ Algorithmus abzuschließen, wird $U_s$ auf $\psi_2$ angewandt:

$$ G = U_S U_w |\psi\rangle= U_S |\psi_2\rangle = |\psi_{final}\rangle= \left[\begin{matrix}- \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & - \frac{1}{2} & \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & \frac{1}{2} & - \frac{1}{2} & \frac{1}{2}\\\frac{1}{2} & \frac{1}{2} & \frac{1}{2} & - \frac{1}{2}\end{matrix}\right] \cdot
\left[\begin{matrix}\frac{1}{2}\\-\frac{1}{2}\\\frac{1}{2}\\\frac{1}{2}\end{matrix}\right]=
\left[\begin{matrix}0\\1\\0\\0\end{matrix}\right]
$$


In [28]:
q_final=U_s*psi_2
#print_latex(q_final)

Die finale Quantenschaltung sieht folgendermaßen aus:

<img src="image/groover_2_Z.png">

Wenn man jetzt das Ergebnis von $\psi_{final}$ misst, bekommt man zu 100% den gesuchten Ergebnisvektor $|01\rangle$. Die Messung erfolgt mit der Sympy Funktion *measure_all*, welche den Ergebnisvektor und die Wahrscheinlichkeit darstellt.


In [13]:
measure_all(q_final)

[(|01>, 1)]

Damit ist der Grover-Algorithmus auf zwei Qubits angewandt und im Detail in Matrizenrechnung in allen drei Schritten, also Superposition, Orakel und Inversionsspiegelung, durchgeführt worden.


#### Bezugnahme auf den Hinweis oben: Qubits mathematisch-physikalischer Darstellung versus IBM Quantum/Qiskit

Wie oben beschrieben entspricht das Ergebnis in Sympy nach der mathematisch-physikalischen Darstellung $|01\rangle$ der IBM Quantum/ Quiskit Darstellung $|10\rangle$.